In [4]:
import tensorflow as tf
import os
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K
from tensorflow.keras.losses import binary_crossentropy
import time

from skimage.io import imread, imshow
from sklearn.metrics import  f1_score, precision_score, recall_score, jaccard_score

In [5]:
if tf.config.list_physical_devices('GPU'):
    print("GPU disponível")
else:
    print("GPU não disponível")

# Aloca memória de forma dinâmica para a GPU
gpu_devices = tf.config.list_physical_devices('GPU')
if gpu_devices:
    tf.config.experimental.set_memory_growth(gpu_devices[0], True)

# Caso queira controlar a quantidade de memória alocada, use:
# tf.config.set_virtual_device_configuration(gpu_devices[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])

GPU não disponível


In [6]:
def dice_coef(y_true, y_pred, smooth=1):
    # Ensure the tensors are of type float32
    y_true_f = K.cast(K.flatten(y_true), 'float32')
    y_pred_f = K.cast(K.flatten(y_pred), 'float32')
    
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def iou_coef(y_true, y_pred, smooth=1):
  intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
  union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
  iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
  return iou

def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - score

def bce_dice_loss(y_true, y_pred):
    return binary_crossentropy(tf.cast(y_true, tf.float32), y_pred) + 0.5 * dice_loss(tf.cast(y_true, tf.float32), y_pred)


### Convert the model

In [7]:
#carrega o modelo
model = tf.keras.models.load_model('models/1final_model_unetoriginal.keras', custom_objects={'bce_dice_loss': bce_dice_loss, 'dice_coef': dice_coef}, safe_mode=False)

In [4]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('TinyML/tflite', 'wb') as f:
        f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\joaod\AppData\Local\Temp\tmpcnunyhqf\assets


INFO:tensorflow:Assets written to: C:\Users\joaod\AppData\Local\Temp\tmpcnunyhqf\assets


Saved artifact at 'C:\Users\joaod\AppData\Local\Temp\tmpcnunyhqf'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 128, 128, 1), dtype=tf.float32, name=None)
Captures:
  2375008312144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2375008311760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2375008311952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2375008312336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2375008312528: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2375008309840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2375008313872: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2375008313104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2375008314832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2375008313296: TensorSpec(shape=(), dtype=tf.resource, name=None

In [5]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_qaware_model = converter.convert()
with open('TinyML/tflite-reduzido', 'wb') as f:
    f.write(tflite_qaware_model)

INFO:tensorflow:Assets written to: C:\Users\joaod\AppData\Local\Temp\tmpnk5rd_ff\assets


INFO:tensorflow:Assets written to: C:\Users\joaod\AppData\Local\Temp\tmpnk5rd_ff\assets


Saved artifact at 'C:\Users\joaod\AppData\Local\Temp\tmpnk5rd_ff'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 128, 128, 1), dtype=tf.float32, name=None)
Captures:
  2375008312144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2375008311760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2375008311952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2375008312336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2375008312528: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2375008309840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2375008313872: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2375008313104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2375008314832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2375008313296: TensorSpec(shape=(), dtype=tf.resource, name=None

## Test the model

### Load the data

In [6]:
# Params
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3
VAL_PATH = 'data_mod/validation/'

In [7]:
def resize(input_image, input_mask):
    input_image = tf.image.resize(input_image, (IMG_WIDTH, IMG_HEIGHT), method="nearest")
    input_mask = tf.image.resize(input_mask, (IMG_WIDTH, IMG_HEIGHT), method="nearest")
    # Convert mask to grayscale
    input_mask = tf.image.rgb_to_grayscale(input_mask)

    return input_image, input_mask 

def normalize(input_image):
    input_image = tf.cast(input_image, tf.float32)
    input_image = input_image / 255.0
    return input_image

In [8]:
VAL_PATH = 'data_mod/validation/'

n_val = len(os.listdir(VAL_PATH+'images'))
X_val = []
Y_val = []

# Load val images
# Load train images
for i in tqdm(range(n_val)):
    # Carrega a imagem e a máscara
    img = imread(VAL_PATH+'images/'+os.listdir(VAL_PATH+'images')[i])[:,:,:IMG_CHANNELS]
    mask = imread(VAL_PATH+'labels/'+os.listdir(VAL_PATH+'labels')[i])
    img, mask = resize(img, mask)
    img = normalize(img)
    mask = normalize(mask)
    X_val.append(img)
    Y_val.append(mask)
    
    # Faz a augmentação e adiciona a imagem flipada, se houver
    # img, mask, is_flip = augment(img, mask)
    # if is_flip:
    #     X_val.append(img)
    #     Y_val.append(mask)

# Converter listas para arrays NumPy
X_val = np.array(X_val)
Y_val = np.array(Y_val)
Y_val = (Y_val > 0.5).astype(int)

100%|██████████| 195/195 [00:07<00:00, 24.46it/s]


### Evaluate the models

In [20]:
def evaluate(preds_threshold):
    dice_values = []
    for ix in range(len(Y_val)):
        dice_value = dice_coef(Y_val[ix].flatten(), preds_threshold[ix].flatten()).numpy()  
        dice_values.append(dice_value)

    dice_mean = np.mean(dice_values)



    f1_values = []
    for ix in range(len(Y_val)):
        f1_value = f1_score(Y_val[ix].flatten(), preds_threshold[ix].flatten(), zero_division=0)
        f1_values.append(f1_value)

    f1_mean = np.mean(f1_values)    

    precision_values = []
    for ix in range(len(Y_val)):
        precision_value = precision_score(Y_val[ix].flatten(), preds_threshold[ix].flatten(), zero_division=0)
        precision_values.append(precision_value)

    precision_mean = np.mean(precision_values)

    recall_values = []
    for ix in range(len(Y_val)):
        recall_value = recall_score(Y_val[ix].flatten(), preds_threshold[ix].flatten(), zero_division=0)
        recall_values.append(recall_value)

    recall_mean = np.mean(recall_values)

    iou_values = []
    for ix in range(len(Y_val)):
        iou_value = jaccard_score(Y_val[ix].flatten(), preds_threshold[ix].flatten())
        iou_values.append(iou_value)

    iou_mean = np.mean(iou_values)

    return dice_mean, f1_mean, precision_mean, recall_mean, iou_mean

def tinyML(model_path):
    gpu_delegate = tf.lite.experimental.load_delegate('TinyML/libtensorflowlite_gpu_delegate.so')
    interpreter = tf.lite.Interpreter(model_path=model_path, experimental_delegates=[gpu_delegate])
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    input_shape = input_details[0]['shape']
    output_shape = output_details[0]['shape']

    preds = []
    start_time = time.time()
    for i in range(n_val):
        input_tensor = X_val[i].reshape(input_shape)
        interpreter.set_tensor(input_details[0]['index'], input_tensor)
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_details[0]['index'])
        preds.append(output_data)
    end_time = time.time()
    inference_time = end_time - start_time

    preds = np.array(preds)
    preds_threshold = (preds > 0.3).astype(np.uint8)

    model_size = os.path.getsize(model_path) / (1024 * 1024)
    eval_values = evaluate(preds_threshold)
    return model_path, model_size, inference_time, eval_values

In [21]:
model_path = 'TinyML/tflite'
result = tinyML(model_path)
with open("resultsV2.txt", "a") as file:
    file.write(f"Model: {result[0]}\n")
    file.write(f"Model Size: {result[1]} MB\n")
    file.write(f"Inference Time: {result[2]} s\n")
    file.write(f"Dice Values: {result[3][0]}\n")
    file.write(f"Jaccard Value: {result[3][1]}\n")
    file.write(f"F1 Value: {result[3][2]}\n")
    file.write(f"Recall Value: {result[3][3]}\n")
    file.write(f"Precision Value: {result[3][4]}\n\n")

OSError: [WinError 193] %1 não é um aplicativo Win32 válido

In [11]:
model_path = 'TinyML/tflite-reduzido'
result = tinyML(model_path)
with open("resultsV2.txt", "a") as file:
    file.write(f"Model: {result[0]}\n")
    file.write(f"Model Size: {result[1]} MB\n")
    file.write(f"Inference Time: {result[2]} s\n")
    file.write(f"Dice Value: {result[3][0]}\n")
    file.write(f"Jaccard Value: {result[3][1]}\n")
    file.write(f"F1 Value: {result[3][2]}\n")
    file.write(f"Recall Value: {result[3][3]}\n")
    file.write(f"Precision Value: {result[3][4]}\n\n")